In [1]:
import pickle
import networkx
from itertools import combinations

creating_pref_attach_graph = False;
creating_pref_attach_reduced_graph = False;
creating_shortest_paths_graph = False;
G1,G2,G3,G4 = None,None,None,None

In [19]:
x = 50000
for _ in range(10):
    x = (x*1.20) + 80000
    print(_,x)

0 140000.0
1 248000.0
2 377600.0
3 533120.0
4 719744.0
5 943692.7999999999
6 1212431.3599999999
7 1534917.6319999998
8 1921901.1583999996
9 2386281.3900799993


### LINKS
Tutorial how to use networkX : <a href="https://networkx.github.io/documentation/stable/tutorial.html#accessing-edges-and-neighbors"> networkx.github.io</a> 

In [2]:
with open('base_data/G_california.p', 'rb') as f:
    G = pickle.load(f)

In [3]:
print("Number of nodes : ", G.number_of_nodes())
print("Number of edges : ", G.number_of_edges())
print("\n10 first nodes :\n", "\t"+ "\n\t".join([str(node) for node in list(G.nodes())[:10]]))
print("\n10 first edges :\n", "\t" +"\n\t".join([str(edge) for edge in list(G.edges())[:10]]))

Number of nodes :  9816
Number of edges :  16424

10 first nodes :
 	hortau
	spiritual-gangster
	brit
	seatninja
	bluedata-software
	village-defense
	netherfire-entertainment
	dubuc-motors
	reflektive
	bundled-bliss

10 first edges :
 	('hortau', 'inv_advantage-capital-partners')
	('hortau', 'inv_avrio-capital')
	('spiritual-gangster', 'inv_m3-ventures')
	('brit', 'inv_shervin-pishevar')
	('brit', 'inv_general-catalyst-partners')
	('brit', 'inv_jim-felding')
	('brit', 'inv_cowboy-ventures')
	('brit', 'inv_marissa-mayer')
	('brit', 'inv_jennifer-hyman')
	('brit', 'inv_ff-angel-llc')


In [4]:
invs = [node for node in list(G.nodes()) if node[:4]=='inv_']
vs = [node for node in list(G.nodes()) if node[:4]!='inv_']
n_invs= len(invs)
n_vs = len(vs)
print("Extracted {0} ventures and {1} investors so {2} (/{3}) nodes ".format(len(invs), len(vs), len(invs)+len(vs),  G.number_of_nodes()))

Extracted 5883 ventures and 3933 investors so 9816 (/9816) nodes 


# 2 - Preferential attachement based on degrees

In this part we are interested in preferential attachement, which states that two nodes are likely to be linked when they are already of *high degree* which means, that those nodes are already having a lot of links (edges). This idea states that usually people link to famous people because of the scarcity of information. Nevertheless, there are big limitations to this information in our case, because one start-up can only be invested by a fistful of investors, and a couple of investor for each round.

In the following we show that **preferential attachement of (venture, inv) is 16 times bigger when indeed a link exists between the two stakeholders**. This results might be discussed because, nodes which are connected in the real graph, because of being connected, are likely to have high degree.

To verify more subtily our result, we should get rid of one existing link, calculate the pref attachement between the two nodes, and then suppose that this link is likely to happen, in the same graph as initial except that special edge. To do that, we should just reduce by one the degree of the two nodes in the measure of metric_pref_attach. The result falls back to **14 times larger**

In [5]:

max_dv_dinv_inverse = ((n_invs * n_vs)**(-1))
                       
def metric_pref_attach(node_v, node_inv, G,  max_dv_dinv_inverse):
    return  max_dv_dinv_inverse * G.degree(node_v) * G.degree(node_inv)
                       
def metric_pref_attach_reduced(node_v, node_inv, G,  max_dv_dinv_inverse):
    return  max_dv_dinv_inverse * (G.degree(node_v) -1) * (G.degree(node_inv)-1)

In [6]:
%%time
if creating_pref_attach_graph :
    print("Creating")
    G2 = networkx.Graph()
    G2.add_nodes_from(G)
    i = 0
    for v_i in vs :
        if not i%1000 :
            print(i)
        i+=1
        for inv_j in invs :
            G2.add_edge(v_i, inv_j, weight = metric_pref_attach(v_i, inv_j, G, max_dv_dinv_inverse))
    pickle.dump( G2, open( "output_data/pref_attach_edges_graph.p", "wb" ) )
else :
    print("Loading G2")
    with open('output_data/pref_attach_edges_graph.p', 'rb') as f:
        G2 = pickle.load(f)

Loading G2
CPU times: user 14.8 s, sys: 31.8 s, total: 46.6 s
Wall time: 47 s


In [7]:
existing_investment_edges = set(G.edges)
all_possible_edges = set(G2.edges)
all_not_existing_investment_edges = all_possible_edges - existing_investment_edges
print("Number of investment edges : ",len(existing_investment_edges))
print("Number of non-invested edges : ", len(all_not_existing_investment_edges))
print("Theoretically, possible edges are Nv x Ninv = ", len(vs)*len(invs))
print("Matching : ", len(existing_investment_edges) + len(all_not_existing_investment_edges) == len(vs)*len(invs))


Number of investment edges :  16424
Number of non-invested edges :  23121415
Theoretically, possible edges are Nv x Ninv =  23137839
Matching :  True


In [9]:
%%time
sum_of_existing_edges_pref_value = 0
sum_of_non_existing_edges_pref_value = 0
for v, inv in existing_investment_edges :
    sum_of_existing_edges_pref_value += G2[v][inv]['weight']
for v, inv in all_not_existing_investment_edges :
    sum_of_non_existing_edges_pref_value += G2[v][inv]['weight']
mean_existing_pref = sum_of_existing_edges_pref_value / len(existing_investment_edges)
mean_non_existing_pref = sum_of_non_existing_edges_pref_value / len(all_not_existing_investment_edges)
print(("When a link exists, preferential attachement is on average {0}\n and {1} "
      + "when not existing").format(mean_existing_pref, mean_non_existing_pref))

When a link exists, preferential attachement is on average 8.037530164718666e-06
 and 4.98511341560131e-07 when not existing


In [10]:
%%time
del G2

In [8]:
%%time
if creating_pref_attach_reduced_graph :
    print("Creating")
    G3 = networkx.Graph()
    G3.add_nodes_from(G)
    i = 0
    for v_i in vs :
        if not i%200 :
            print(i)
        i+=1
        for inv_j in invs :
            if (v_i,inv_j) in existing_investment_edges :
                G3.add_edge(v_i, inv_j, weight = metric_pref_attach_reduced(v_i, inv_j, G, max_dv_dinv_inverse))
            else : 
                G3.add_edge(v_i, inv_j, weight = metric_pref_attach(v_i, inv_j, G, max_dv_dinv_inverse))
    pickle.dump( G3, open( "output_data/pref_attach_reduced_edges_graph.p", "wb" ) )
else :
    print("Loading G3")
    with open('output_data/pref_attach_reduced_edges_graph.p', 'rb') as f:
        G3 = pickle.load(f)

Loading G3
CPU times: user 19.5 s, sys: 1min 17s, total: 1min 37s
Wall time: 1min 38s


In [11]:
%%time
sum_of_existing_edges_pref_value = 0
sum_of_non_existing_edges_pref_value = 0
for v, inv in existing_investment_edges :
    sum_of_existing_edges_pref_value += G3[v][inv]['weight']
for v, inv in all_not_existing_investment_edges :
    sum_of_non_existing_edges_pref_value += G3[v][inv]['weight']
mean_existing_pref = sum_of_existing_edges_pref_value / len(existing_investment_edges)
mean_non_existing_pref = sum_of_non_existing_edges_pref_value / len(all_not_existing_investment_edges)
print(("Method #2 : \nWhen a link exists, preferential attachement is on average {0}\n and {1} "
      + "when not existing").format(mean_existing_pref, mean_non_existing_pref))

Method #2 : 
When a link exists, preferential attachement is on average 6.823317708936458e-06
 and 4.98511341560131e-07 when not existing


In [12]:
%%time
del G3

# 3 - Measuring the connection between two nodes 

* Hitting time or Commute time (based on random walks).
* Personalized Pagerank.
* G4 - Shortest path length (or geodesic distance) Note that in weighted graph u can use **Djikstra**

### 3.1 - Shortest Paths

In [18]:
%%time
Gtemp = G
if creating_shortest_paths_graph :
    print("Creating shortest paths")
    G4 = networkx.Graph()
    G4.add_nodes_from(G)
    i = 1
    for v_i in vs :
        for inv_j in invs :
            stored_edge = False
            if (v_i, inv_j) in existing_investment_edges :
                stored_edge = True
                Gtemp.remove_edge(v_i,inv_j)
            try :
                G4.add_edge(v_i, inv_j, weight = networkx.shortest_path_length(Gtemp, v_i, inv_j))
            except :
                None
            if stored_edge :
                Gtemp.add_edge(v_i,inv_j)
        if not i%100 :
            print(i)
        i+=1
    print("Dumping")
    pickle.dump( G4, open( "output_data/creating_shortest_paths_graph.p", "wb" ) )
else :
    print("Loading G4")
    with open('output_data/creating_shortest_paths_graph.p', 'rb') as f:
        G4 = pickle.load(f)

Loading G4
CPU times: user 11.5 s, sys: 14.5 s, total: 26 s
Wall time: 26.9 s


In [19]:
%%time
sum_of_existing_edges_shortest_path = 0
sum_of_non_existing_edges_shortest_path = 0

print("Calculating sets # 1")
connected_in_G4 = set(G4.edges)
not_connected_in_G4 = all_possible_edges - connected_in_G4
invested = set(G.edges)
not_invested = all_possible_edges - invested

print("Calculating sets # 2")
connected_not_invested_couples =  connected_in_G4.intersection(not_invested)
connected_invested_couples = connected_in_G4.intersection(invested)
not_connected_not_invested_couples = not_connected_in_G4.intersection(not_invested)
not_connected_invested_couples = not_connected_in_G4.intersection(invested)
print(len(connected_not_invested_couples ))
print(len(connected_invested_couples ))
print(len(not_connected_not_invested_couples))
print(len(not_connected_invested_couples))

print("Calculating metrics")
for v, inv in connected_invested_couples :
    sum_of_existing_edges_shortest_path += G4[v][inv]['weight']
        
for v, inv in connected_not_invested_couples :
    sum_of_non_existing_edges_shortest_path += G4[v][inv]['weight']

print("sum_of_existing_edges_shortest_path",sum_of_existing_edges_shortest_path)
print("sum_of_non_existing_edges_shortest_path",sum_of_non_existing_edges_shortest_path)


Calculating sets # 1
Calculating sets # 2
14256662
11643
8864753
4781
Calculating metrics
sum_of_existing_edges_shortest_path 43239
sum_of_non_existing_edges_shortest_path 79747880
CPU times: user 1min 9s, sys: 20.1 s, total: 1min 29s
Wall time: 1min 36s


### Conclusions :

**4 781** investment (over **16 424**) are made between two nodes which are impossible to connect otherwise.

In the **23 121 415** candidates connexion, **14 256 662** connexion already exist inderictly (a shortest_path exists), leaving out **8 864 753** (v,inv) that are not linked at all.

In other words, when two nodes are in someway connected (not investment) 

### Bla

X|Invested|Not Invested|
--|--|--|
connected | 11 643<br/>mean:**3.71**| 14 256 662 <br/> mean:**5.59**|
not connected | 4 781<br/> p=0.41 | 8 864 753 <br/> p = 0.383|

Nous avons les probabilités p( connected | invest ) = 0.59 et p ( connected | not_invested ) = 0.62
Avec **une formule de Bayes**, on peut retrouver p( invested | connected) et l'autre aussi.

On peut obtenir une formule plus fine en utilisant des variables gaussiennes.

On voit que si deux noeuds vont se connecter dans le futur, qu'ils soient lié ou pas ne change pas grand chose.
* En revanche, s'ils sont liés avec une longueur < 4 indique qu'il y a plus de chances qu'il soit investit.
* Et invesement, s'ils sont liés avec une longueur > 4 indique qu'il y a plus de chance qu'il ne le soit pas.

In [15]:
del G4